In [100]:
import anthropic

In [101]:
from dotenv import load_dotenv

In [102]:
load_dotenv()

True

In [103]:
client_anthropic = anthropic.Anthropic(api_key = os.getenv('CLAUDE_API_KEY'))

In [104]:
import os

In [105]:
import json

In [106]:
from supabase import create_client, Client

In [107]:
supabase_api_key = os.environ.get("SUPBASE_KEY")
supabase_url = os.environ.get("SUPABASE_URL")

In [108]:
supabase: Client = create_client(supabase_url, supabase_api_key)

In [109]:
researched_data = [{'event_id': '299_2026-02-08',
  'news_date': '2026-02-08',
  'output': '- Verified: The earbuds rumor centers on an internal code name for OpenAI’s first consumer hardware device being Sweetpea. Multiple credible outlets report this codename as part of the leak cycle about the device. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Verified: Several credible outlets note that the project is not officially confirmed and remains a leak/rumor at this time. TechCrunch explicitly describes the details as coming from leaks and “tipsters,” not an official OpenAI confirmation; Axios likewise characterizes the information as rumors shared at Davos. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Verified: The same reporting that mentions Sweetpea also references a potential second consumer device codenamed Gumdrop in the rumor mill. ([axios.com](https://www.axios.com/2026/01/23/openai-device-earbuds-sweetpea-altman))\n\n- Added confirmed hardware context: OpenAI disclosed at Davos that it is building a new AI device, though details remain scarce. Timeline given by OpenAI officials in these discussions points to a likely release in the second half of 2026. ([axios.com](https://www.axios.com/2026/01/23/openai-device-earbuds-sweetpea-altman))\n\n- Confirmed: OpenAI’s prior hardware efforts are linked to its 2025 acquisition of Jony Ive’s design firm io (the deal was reported to be around $6.5 billion). Ive is associated with leading the hardware design work for the product family. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Confirmed: OpenAI’s hardware plans discussed publicly have included a screen-free, pocketable device concept as part of the early product narrative. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Confirmed: Reported design descriptions for the Sweetpea concept include a main unit described as metal and egg-shaped, with two pill-shaped modules that would sit behind the ear. These details appear in multiple outlets drawing on the same leaks. ([indianexpress.com](https://indianexpress.com/article/technology/tech-news-technology/openai-x-jony-ive-wireless-earbuds-unique-design-launch-2026-10471143/))\n\n- Confirmed (design details corroborated by multiple outlets): The rumored hardware would reportedly use a 2-nanometer-class processor (with Exynos as a favored reference) for on-device AI tasks, aligned with the “phone-like” compute ambitions discussed in the leaks. ([indianexpress.com](https://indianexpress.com/article/technology/tech-news-technology/openai-x-jony-ive-wireless-earbuds-unique-design-launch-2026-10471143/))\n\n- Confirmed (manufacturing partner context): Reports mention Foxconn as a manufacturing partner for the OpenAI devices, with some mention of Luxshare as a possibility; this is described in the leakage coverage and echoed by outlets citing Asian reporting. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Confirmed: First-year shipment targets cited in the leaks are in the 40–50 million units range, with September 2026 as a possible launch window in at least some outlets. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\n- Confirmed (scope/roadmap detail in leaks): Some leakage coverage claims OpenAI is planning a broader hardware roadmap (up to five devices by 2028), with Sweetpea at the front of that lineup. This detail appears in multiple leak-focused outlets. ([androidauthority.com](https://www.androidauthority.com/openai-earbuds-3631925/))\n\n- Not corroborated by credible outlets (branding name caveat): The claim that the consumer-facing name will be “Dime” lacks confirmation from established major outlets. Some blogs and tech sites have floated “Dime” as branding based on patent filings or rumor chatter, but there is no widely credited report in outlets like TechCrunch, Axios, or The Indian Express confirming “Dime.” Examples of circulating but less-established sources include Gizmochina and other aggregators. ([gizmochina.com](https://www.gizmochina.com/2026/02/07/openais-first-hardware-device-ai-earbuds-strategy-shift-and-launch-timeline/?utm_source=openai))\n\n- Note on the current status of confirmation: The available reporting from January–February 2026 consistently describes the information as leaks/rumors rather than official confirmation; OpenAI executives acknowledged work on a hardware device but kept specifics (design, branding, features) under wraps. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\nSources referenced above (examples of main reporting):\n- TechCrunch: OpenAI aims to ship its first device in 2026, and it could be earbuds (codename Sweetpea; design, chip, and unit targets discussed). ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n- Indian Express: OpenAI’s first product could be ChatGPT-powered earbuds; codename Sweetpea; design details and 2nm Samsung Exynos chip mentioned. ([indianexpress.com](https://indianexpress.com/article/technology/tech-news-technology/openai-x-jony-ive-wireless-earbuds-unique-design-launch-2026-10471143/))\n- Android Authority: OpenAI earbuds leak with Sweetpea codename; design description; BOM/production cost hints; September launch target; five-device roadmap mentioned. ([androidauthority.com](https://www.androidauthority.com/openai-earbuds-3631925/))\n- Axios: OpenAI’s secret device coming; Davos timeline; mentions Sweetpea and Gumdrop leaks; timeline for second half of 2026. ([axios.com](https://www.axios.com/2026/01/23/openai-device-earbuds-sweetpea-altman))\n- The Verge: OpenAI hardware prototype discussions tied to Jony Ive and the hardware program (context around the hardware push). ([theverge.com](https://www.theverge.com/news/827607/openai-hardware-prototype-chatgpt-jony-ive-sam-altman?utm_source=openai))\n- (Caveat on Dime branding) Gizmochina / other aggregators noting a claimed consumer name Dime; these are not widely corroborated by major outlets. ([gizmochina.com](https://www.gizmochina.com/2026/02/07/openais-first-hardware-device-ai-earbuds-strategy-shift-and-launch-timeline/?utm_source=openai))\n\nNotes:\n- All statements above about Sweetpea, the rumored features, and the general timeline are based on published reporting as of February 8, 2026; several items (notably the consumer-name “Dime”) are not yet verified by major, high-profile outlets and are reported only by a subset of non-primary sources. ([techcrunch.com](https://techcrunch.com/2026/01/21/openai-aims-to-ship-its-first-device-in-2026-and-it-could-be-earbuds/))\n\nIf you’d like, I can add direct excerpts or align these points to a formal, source-by-source bibliography.'}]

In [232]:
result = supabase.table('research_assistant')\
    .select('id, event_id, news_date, output')\
.gte('news_date', '2026-02-12')\
.lte('news_date', '2026-02-15')\
    .order('created_at', desc=False)\
    .execute()

In [233]:
result.data

[]

In [234]:
final_summary_1 = []

In [235]:
def claude_article_builder(researched_data):
    final_news_article = []
    for i in researched_data:
        response = client_anthropic.messages.create(
            model = "claude-sonnet-4-5",
            max_tokens = 800,
            messages = [{
                "role": "user",
                "content":  f"""
                You are a tech journalist at Krux — a news platform that helps GenZ stay informed about AI & tech in under 60 seconds.

                CRITICAL CONSTRAINT: The article body MUST be 85-100 words (excluding headline). Count carefully before finalizing.


                TASK: Write a news article using ONLY the information provided by our research assistant.
                
                Follow this structure:
                
                === STEP 1: HEADLINE (Under 10 words) ===
                Objective: Hook the reader instantly
                - Punchy, attention-grabbing and clickbait
                - Focus on the most surprising/important angle
                
                **ARTICLE BODY** (85-100 words TOTAL)

                Opening (20-30 words):
                - If rumour: clarify in first sentence, then proceed assuming the things stated by research assistant are facts
                - Lead with the core news hook
                
                Middle (40-50 words):
                - Sharp, conversational, slightly witty tone
                - Present key facts from research assistant
                - Use most/all provided information
                - NO links or source references in body
                
                Closing (20-25 words):
                - Forward-looking strategic perspective
                - What could this mean for industry/users/tech landscape?
                - Frame as analysis, NOT opinion
                - Grounded in facts presented
                
                === VERIFICATION CHECKLIST ===
                Before outputting, verify:
                ☑ Article body is 85-100 words (count manually)
                ☑ Headline is under 10 words
                ☑ No URLs in article body
                ☑ All key facts from research included
                
                HARD CONSTRAINTS:
                - Total article: 100 words max (headline excluded). If the Article is NOT 100 words, re-generate it to meet the 100 word
                requirement.
                - Sources with URLs listed at end under "Sources:"

                
                OUTPUT FORMAT NEEDS TO BE IN A STRICT JSON FORMAT:
                {{"headline": "...", "output": "...", "sources": [{{"name": "...", "url": "..."}}]}}

                Here is the information we have gathered from the research assistant:
                
                {i['output']}
            """
            }]
        )

        final_output = response.content[0].text

        cleaned = final_output.strip()

        if cleaned.startswith("```"):
            cleaned = cleaned.split("\n", 1)[1].rsplit("```", 1)[0].strip()

        article_json = json.loads(cleaned)

        article_json['model_provider'] = 'claude'
        article_json['news_date'] = i['news_date']
        article_json['event_id'] = i['event_id']

        print(article_json)

        final_summary_1.append(article_json)

        

        # save_article(article_json)
        

    #     final_news_article.append(json_format)

    # return final_news_article

In [210]:
claude_article_builder(research)

{'headline': 'Amazon Plotting AI Content Marketplace to Rival Microsoft', 'output': 'Amazon is reportedly building an AI content marketplace where publishers can license their work to AI developers. Internal AWS slides link the platform to Bedrock and Quick Suite, positioning it as a direct competitor to Microsoft\'s Publisher Content Marketplace. Amazon declined to comment, saying it has "nothing specific to share." The plan emerged ahead of an AWS conference, with no confirmed launch timeline. Publishers see it as a potential sustainable revenue stream amid ongoing battles over AI training data rights and licensing fees. The marketplace could reshape how media companies monetize content in the AI era, offering a scalable alternative to one-off licensing deals.', 'sources': [{'name': 'TechCrunch', 'url': 'https://techcrunch.com/2026/02/10/amazon-may-launch-a-marketplace-where-media-sites-can-sell-their-content-to-ai-companies/'}, {'name': 'Investing.com', 'url': 'https://www.investing

In [92]:
def save_article(article_json):
    supabase.table('hundred_word_articles').insert({
        'event_id': article_json['event_id'],
        'model_provider': article_json['model_provider'],
        'news_date': article_json['news_date'],
        'sources': json.dumps(article_json['sources']),
        'output': article_json['output'],
        'headline': article_json['headline']
    }).execute()

In [212]:
for i in final_summary:
    save_article(i)
    print("✔️ Saved!")

✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!
✔️ Saved!


In [228]:

fail = []
pas = 0

for i in final_summary:
    words = i['output'].split()
    word_count = len(words)
    if word_count > 100:
        print(f"Fail: {word_count}")
        fail.append(word_count)
    else:
        print(f"Pass: {word_count}")
        pas +=1

print(f"Fail: {sum(fail)/len(fail)}")



Fail: 102
Fail: 104
Fail: 108
Fail: 112
Fail: 110
Pass: 100
Fail: 107
Fail: 101
Fail: 113
Fail: 121
Fail: 108
Fail: 122
Fail: 108
Fail: 101
Pass: 99
Pass: 91
Fail: 104
Fail: 105
Fail: 102
Fail: 116
Pass: 94
Fail: 116
Pass: 95
Fail: 106
Fail: 120
Fail: 109
Fail: 107
Fail: 106
Fail: 112
Pass: 92
Fail: 124
Fail: 106
Fail: 107
Pass: 93
Fail: 103
Fail: 107
Fail: 119
Fail: 117
Fail: 104
Fail: 103
Fail: 116
Pass: 94
Pass: 92
Pass: 99
Fail: 105
Fail: 106
Fail: 109.36111111111111


In [223]:
import statistics as stats